In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from matplotlib import image as img
from matplotlib import pyplot as plt
import json

import torch
from torch.nn import Module
from torch.utils.data import Dataset, DataLoader
!pip install torchsummary
from torchsummary import summary
from torchvision import transforms, models

In [ ]:
BASE_DIR = '/kaggle/input/eurosat-dataset/EuroSAT'
BATCH_SIZE = 128
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_EPOCHS = 5

## Loading data

In [ ]:
test_df = pd.read_csv(os.path.join(BASE_DIR, 'test.csv'))
train_df = pd.read_csv(os.path.join(BASE_DIR, 'train.csv'))
validation_df = pd.read_csv(os.path.join(BASE_DIR, 'validation.csv'))

train_df

In [ ]:
with open(os.path.join(BASE_DIR, 'label_map.json'), 'r') as f:
    json_data = f.read()

label_map = json.loads(json_data)
label_map

In [ ]:
train_df['Label'].value_counts()

## Dataset and Dataloader

In [ ]:
train_transforms = transforms.Compose([     
    transforms.ColorJitter(brightness=[0.5, 1.5], contrast=[0.8, 1.2], saturation=[0.8, 1.2]),
    transforms.Resize((224, 224), interpolation=Image.NEAREST),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_val_transforms = transforms.Compose([            
    transforms.Resize((224, 224), interpolation=Image.NEAREST),
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
class Dataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.transforms = transforms
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        
        path = os.path.join(BASE_DIR, self.df['Filename'][idx])
        
        image = Image.open(path)
        image = self.transforms(image)
    
        label = self.df['Label'][idx]
        bin_label = 0 if label in [0, 1, 2, 5, 6, 8, 9] else 1
        
        return image, bin_label
    
train_set = Dataset(train_df, transforms=train_transforms)
val_set = Dataset(validation_df, transforms=test_val_transforms)
test_set = Dataset(test_df, transforms=test_val_transforms)

In [ ]:
train_loader = DataLoader(
    train_set,
    batch_size=BATCH_SIZE,
    shuffle=True,
)

val_loader = DataLoader(
    val_set,
    batch_size=BATCH_SIZE,
    shuffle=True,
)

test_loader = DataLoader(
    test_set,
    batch_size=BATCH_SIZE,
    shuffle=True
)

### Sample images

In [ ]:
for image_batch, label_batch in val_loader:
    image_batch, label_batch = image_batch[:4], label_batch[:4] 
    fig, axes = plt.subplots(1, 4, figsize=(64, 64))
    axes = [axes]
    for i, data in enumerate(zip(axes[0], image_batch, label_batch)):
        ax, image, label = data
        ax.imshow(image.permute(1, 2, 0))
        ax.axis('off')
        label = 'urbanized' if label.item() == 1 else 'not urbanized'
        ax.set_title(label, fontsize=50)

    plt.tight_layout()
    plt.show()
    break

## Model

In [ ]:
class Model(Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.base = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
        
        self.hidden1 = torch.nn.Linear(1000, 512)
        self.bn1 = torch.nn.BatchNorm1d(512)
        self.hidden2 = torch.nn.Linear(512, 256)
        self.bn2 = torch.nn.BatchNorm1d(256)
        self.hidden3 = torch.nn.Linear(256, 128)
        self.bn3 = torch.nn.BatchNorm1d(128)
        self.out = torch.nn.Linear(128, 1)
        self.sig = torch.nn.Sigmoid()
        
        self.dropout = torch.nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.base(x)
        x = self.hidden1(x)
        x = self.dropout(x)
        x = self.bn1(x)
        x = self.hidden2(x)
        x = self.dropout(x)
        x = self.bn2(x)
        x = self.hidden3(x)
        x = self.dropout(x)
        x = self.bn3(x)
        x = self.out(x)
        out = self.sig(x)
        
        return out

model = Model()
model = model.to(DEVICE)
summary(model, (3, 224, 224))

In [ ]:
criterion = torch.nn.BCELoss()

optimizer = torch.optim.Adam(model.parameters())

In [ ]:
def accuracy(pred, targ):
    pos = 0
    for pred, targ in zip(pred, targ):
        if float(round(pred.item())) == targ.item():
            pos += 1
    return pos/BATCH_SIZE

## Training

In [ ]:
train_losses = []
val_losses = []
train_accs = []
val_accs = []

for epoch in range(NUM_EPOCHS):
    print('\n ----------------------------------')
    print(f'EPOCH: {epoch + 1}/{NUM_EPOCHS}')
        
    train_loss = 0.0
    train_acc = 0.0
    validation_loss = 0.0
    val_acc = 0.0
    
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        out = model(images)
        labels = labels.unsqueeze(1)
        labels = labels.float()
        
        loss = criterion(out, labels)
        loss.backward()
        
        optimizer.step()
        
        train_loss += loss.item()
        train_acc += accuracy(out, labels)
        
    train_loss = train_loss/len(train_loader)
    train_losses.append(train_loss)
    print(f'train loss: {round(train_loss, 3)}')
    
    train_acc = train_acc/len(train_loader)
    train_accs.append(train_acc)
    print(f'train accuracy: {round(train_acc, 3)}\n')
    
    model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            
            out = model(images)
            
            labels = labels.unsqueeze(1)
            labels = labels.float()
            
            loss = criterion(out, labels)
            
            validation_loss += loss.item()
            val_acc += accuracy(out, labels)
            
    validation_loss = validation_loss/len(val_loader)
    val_losses.append(validation_loss)
    print(f'validation loss: {round(validation_loss, 3)}')
    
    val_acc = val_acc/len(val_loader)
    val_accs.append(val_acc)
    print(f'validation accuracy: {round(val_acc, 3)}')
    

## Test

In [ ]:
test_loss = 0.0
test_acc = 0.0

for images, loader in test_loader:
    images, labels = images.to(DEVICE), labels.to(DEVICE)
    
    out = model(images)
    
    labels.unsqueeze(1)
    lables = labels.float()
    
    loss = criterion(out, labels)
    
    test_loss += loss.item()
    test_acc += accuracy(out, labels)
    
test_loss = test_loss/len(test_loader)
test_acc = test_acc/len(test_loader)
    
print("TEST")
print(f'Loss: {test_loss}')
print(f'Accuracy: {test_acc}')

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_accs, label='Train accuracy')
plt.plot(val_accs, label='Validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()vv

### Sample predictions

In [ ]:
for image_batch, label_batch in train_loader:
    image_batch, label_batch = image_batch[:4], label_batch[:4] 
    fig, axes = plt.subplots(1, 4, figsize=(64, 64))
    axes = [axes]
    for i, data in enumerate(zip(axes[0], image_batch, label_batch)):
        ax, image, label = data
        
        out = model(image )
        ax.imshow(image.permute(1, 2, 0))
        ax.axis('off')
        label = 'urbanized' if label.item() == 1 else 'not urbanized'
        pred = 'urbanized' if round(model(image)) == 1 else 'not urbanized'
        ax.set_title(f'Predicted: {pred} | True: {label}', fontsize=50)

    plt.tight_layout()
    plt.show()
    break